In [2]:
import pandas as pd
import numpy as np
import sys
import os
import geopandas as gpd

In [68]:
apart_path = "G:/マイドライブ/akiyamalab/オープン住宅地図/code_data/feture_engineering/housing_and_land_survey/6/g154_3_共同住宅の棟数.xlsx"
apart = pd.read_excel(apart_path,  skiprows=8)

target_area = '15212'
feature_path = f"G:/マイドライブ/akiyamalab/オープン住宅地図/dev_2025/extrapolation/feature_engineering/apart/{target_area}_apart_features.parquet"

In [87]:
predict_path = "G:/マイドライブ/akiyamalab/オープン住宅地図/dev_2025/extrapolation/predict/apart/15212/15212_apart.parquet"
predict = gpd.read_parquet(predict_path)

In [65]:
bldg_path = f"G:/マイドライブ/akiyamalab/オープン住宅地図/dev_2025/extrapolation/feature_engineering/apart/{target_area}_apart.parquet"
bldg = gpd.read_parquet(bldg_path)

In [41]:
target_area = '15581'
# feature[feature['KEY_CODE'].str.startswith(target_area)]

In [5]:
features_path = "G:/マイドライブ/akiyamalab/オープン住宅地図/dev_2025/extrapolation/feature_engineering/detached/15212_features.parquet"
features = gpd.read_parquet(features_path)  

In [69]:
apart['CITY_CODE'] = apart['地域区分'].str[:5]
target_pref_code = target_area[:2]  # 都道府県コードは最初の2桁
apart = apart[apart['地域区分'].str.startswith(target_pref_code)].copy()

feature['CITY_CODE'] = feature['KEY_CODE'].str[:5]
feature = feature[feature['KEY_CODE'].str.startswith(target_area)].copy()

In [80]:
apart['1_長屋建'] = pd.to_numeric(apart['1_長屋建'], errors='coerce').fillna(0)
apart['2_共同住宅'] = pd.to_numeric(apart['2_共同住宅'], errors='coerce').fillna(0)
apart['1_長屋建'] = apart['1_長屋建'].replace('-', 0).astype(int)
apart['2_共同住宅'] = apart['2_共同住宅'].replace('-', 0).astype(int)

apart = apart[apart['地域区分'].str.startswith(target_area)].copy()

apart = apart[apart['建物の構造'] == '0_総数'].copy()

apart_buildings_total = apart['1_長屋建'] + apart['2_共同住宅']

In [86]:
apart_buildings_total*0.0081

1224    4.617
dtype: float64

In [89]:
predict['presumed_detached'].value_counts()

presumed_detached
0    47443
1      497
Name: count, dtype: int64

In [44]:
apart_city_codes = apart['CITY_CODE'].unique()
feature_city_codes = feature['CITY_CODE'].unique()

In [46]:
# 住調にデータが存在する市区町村
cities_with_jucho = set(apart_city_codes)
# 住調にデータが存在しないが、国調には存在する市区町村
cities_without_jucho = set(feature_city_codes) - cities_with_jucho

In [53]:
feature

,KEY_CODE,PREF_NAME,CITY_NAME,S_NAME,geometry,年少人口,生産年齢人口,老年人口,出生時から,1年未満,...,20年以上,居住期間「不詳」,00_総数,01_500万円未満,07_500～1000万円未満,09_1000以上,一戸建,長屋建,共同住宅,CITY_CODE
4657,152120010,新潟県,村上市,羽黒町,"POLYGON ((139.47844 38.21939, 139.47900 38.219...",0.099083,0.528440,0.363303,0.148624,0.025688,...,0.469725,0.009174,1.0,0.697226,0.254889,0.047886,174.0,0.0,48.0,15212
4658,152120020,新潟県,村上市,長井町,"POLYGON ((139.47855 38.22162, 139.47856 38.221...",0.153153,0.441441,0.405405,0.306306,0.018018,...,0.423423,0.000000,1.0,0.677736,0.267740,0.054525,41.0,0.0,0.0,15212
4659,152120030,新潟県,村上市,上町,"POLYGON ((139.47912 38.22312, 139.47901 38.222...",0.116505,0.514563,0.368932,0.281553,0.019417,...,0.398058,0.000000,1.0,0.675778,0.272720,0.051503,32.0,0.0,8.0,15212
4660,152120040,新潟県,村上市,大町,"POLYGON ((139.47941 38.22445, 139.47912 38.223...",0.084507,0.464789,0.450704,0.197183,0.056338,...,0.619718,0.028169,1.0,0.667975,0.274818,0.057207,26.0,0.0,0.0,15212
4661,152120050,新潟県,村上市,小町,"POLYGON ((139.47856 38.22727, 139.47885 38.227...",0.157407,0.518519,0.324074,0.398148,0.046296,...,0.324074,0.000000,1.0,0.670683,0.275012,0.054305,32.0,0.0,0.0,15212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4965,152123010,新潟県,村上市,大毎,"POLYGON ((139.57246 38.46417, 139.57228 38.463...",0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,15212
4966,152123020,新潟県,村上市,碁石,"POLYGON ((139.57577 38.47100, 139.57549 38.470...",0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,15212
4967,152123030,新潟県,村上市,芦谷,"POLYGON ((139.51184 38.47931, 139.51264 38.479...",0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,15212
4968,152123040,新潟県,村上市,越沢,"POLYGON ((139.49434 38.45026, 139.49438 38.450...",0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,15212


In [47]:
cities_with_jucho

{'01209',
 '08210',
 '25212',
 '11233',
 '20000',
 '40213',
 '26204',
 '12239',
 '23106',
 '33211',
 '08207',
 '35210',
 '25202',
 '26205',
 '27217',
 '28218',
 '39212',
 '11209',
 '03207',
 '12100',
 '23111',
 '46206',
 '32000',
 '32205',
 '43212',
 '27210',
 '27118',
 '21221',
 '04208',
 '23210',
 '16210',
 '41208',
 '14102',
 '27213',
 '44205',
 '11108',
 '12217',
 '12103',
 '17206',
 '40207',
 '47208',
 '46218',
 '19208',
 '18207',
 '09215',
 '03201',
 '12101',
 '25209',
 '16000',
 '27144',
 '01236',
 '12106',
 '41207',
 '14118',
 '23224',
 '27116',
 '14137',
 '34207',
 '13208',
 '22212',
 '27207',
 '12224',
 '23222',
 '20204',
 '43206',
 '13120',
 '11105',
 '22000',
 '28102',
 '28212',
 '17204',
 '24214',
 '26211',
 '01101',
 '06207',
 '06212',
 '19212',
 '27228',
 '37203',
 '38206',
 '26108',
 '40133',
 '12221',
 '40103',
 '46221',
 '10203',
 '23234',
 '32204',
 '14214',
 '11203',
 '27226',
 '15107',
 '24203',
 '45000',
 '08230',
 '07205',
 '09203',
 '12104',
 '22203',
 '12207',


In [48]:
cities_without_jucho

{'15581'}

In [18]:
apart_city_codes

array(['00000', '01000', '01100', ..., '47213', '47214', '47215'],
      dtype=object)

In [15]:
feature_city_codes

array(['15212'], dtype=object)

In [24]:
apart

,地域識別コード,地域区分,建物の構造,1_長屋建,11_長屋建_２住宅,12_長屋建_３住宅,13_長屋建_４住宅,14_長屋建_５住宅以上,１むね当たり住宅数（長屋建）,2_共同住宅,...,22_共同住宅_10～19住宅,23_共同住宅_20～29住宅,24_共同住宅_30～39住宅,25_共同住宅_40住宅以上,１むね当たり住宅数（共同住宅）,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,a,00000_全国,0_総数,492300,248500,59200,84900,99600,3.45,2381000,...,491900,156400,79700,131000,12.72,113800,32200,37700,43900,44.73
1,a,00000_全国,1_木造,356200,195500,45200,54500,61100,3.25,721800,...,95000,5700,600,100,6.12,3000,2700,300,0,9.36
2,a,00000_全国,2_非木造,136100,53100,14100,30500,38500,3.97,1659200,...,396900,150700,79100,130800,15.59,110800,29500,37500,43800,45.67
3,a,01000_北海道,0_総数,37700,25300,2500,6500,3300,2.8,129100,...,24800,7200,3500,5000,10.69,4500,1400,1700,1500,36.45
4,a,01000_北海道,1_木造,30600,22300,2100,3900,2200,2.64,76900,...,9200,400,0,0,5.96,300,300,0,0,10.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3112,2,47214_宮古島市,1_木造,10,-,-,10,0,4.29,30,...,10,-,-,-,9.33,-,-,-,-,-
3113,2,47214_宮古島市,2_非木造,80,20,10,30,30,4.86,1360,...,330,100,30,20,10.17,50,40,10,0,11.49
3114,2,47215_南城市,0_総数,60,40,10,-,10,2.59,500,...,160,50,20,0,10.52,-,-,-,-,-
3115,2,47215_南城市,1_木造,-,-,-,-,-,-,0,...,-,-,-,-,5.26,-,-,-,-,-
